In [3]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [4]:
w, h = 20, 20
base_path = os.path.join('drive/MyDrive/dataset', '*/*.npz') #colab google drive
#base_path = os.path.join('dataset', '*/*.npz') 
# create_dataset으로 형성된 data

file_list = glob(base_path)

x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

# CNN사용을 위한 형 변환
x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

100%|██████████| 2130/2130 [10:52<00:00,  3.26it/s]


(1244572, 20, 20, 1) (1244572, 400)
(311144, 20, 20, 1) (311144, 400)


In [5]:
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1)),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(64, 7, activation='relu', padding='same'),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    # (20,20,1)-> (,400)의 원래형태를 만들어준다.
    # 가로 * 세로 = 400
    layers.Reshape((h * w,)),
    layers.Activation('sigmoid')
    # output이 0/1 둘중 하나 (sigmoid)
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 20, 64)        3200      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 128)       401536    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 256)       1605888   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       1605760   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 64)        401472    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 1)         65        
_________________________________________________________________
reshape (Reshape)            (None, 400)               0

In [6]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), 
                        monitor='val_acc', 
                        verbose=1, 
                        save_best_only=True, 
                        mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', 
                          factor=0.2, 
                          patience=5, 
                          verbose=1, 
                          mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

Epoch 1/10
4862/4862 [==============================] - 686s 133ms/step - loss: 0.0141 - acc: 0.2514 - val_loss: 0.0064 - val_acc: 0.5340

Epoch 00001: val_acc improved from -inf to 0.53395, saving model to ./models/20210325_111645.h5
Epoch 2/10
4862/4862 [==============================] - 653s 134ms/step - loss: 0.0060 - acc: 0.5535 - val_loss: 0.0055 - val_acc: 0.5818

Epoch 00002: val_acc improved from 0.53395 to 0.58177, saving model to ./models/20210325_111645.h5
Epoch 3/10
4862/4862 [==============================] - 655s 135ms/step - loss: 0.0052 - acc: 0.5981 - val_loss: 0.0052 - val_acc: 0.6028

Epoch 00003: val_acc improved from 0.58177 to 0.60279, saving model to ./models/20210325_111645.h5
Epoch 4/10
4862/4862 [==============================] - 660s 136ms/step - loss: 0.0048 - acc: 0.6299 - val_loss: 0.0050 - val_acc: 0.6170

Epoch 00004: val_acc improved from 0.60279 to 0.61698, saving model to ./models/20210325_111645.h5
Epoch 5/10
4862/4862 [=============================

In [8]:
y_pred = model.predict(np.expand_dims(x_val[i], axis=0)).squeeze()
y_pred = y_pred.reshape((h, w))

y, x = np.unravel_index(np.argmax(y_pred), y_pred.shape)

print(x, y, y_pred[y, x])

3 17 0.655562
